# Jax・Brax・HaikuでGPU引きこもり学習
> 強化学習若手の会 Advent Calendar 2021 18日目

- toc: true 
- badges: true
- comments: true
- categories: [ja]

# 0. 時勢のあいさつとか

> Note: このブログは[強化学習若手の会 Advent Calendar 2021](https://adventar.org/calendars/6362) 18日目の記事として書かれました

こんにちは。
コロナ禍も終わりが見えつつある（と思ったら変異株が...）2021年もあとわずか。寒さも厳しくなってきましたが、皆さん如何お過ごしでしょうか。
僕は沖縄にいるのであまり寒くはないし虫歯の治療中で奥歯に穴が空いている以外はおおむね元気です。

[若手の会](https://young-reinforcement.github.io/)のアドベントカレンダーということで、国内でのコミュニティの活動について、最初に総括してみたいと思います。

- 若手の会では模倣学習の勉強会をしましたが、結局2回しか続きませんでした。
- 苦手の会のもくもく会はフリスビーやマラソンの練習とバッティングしてやらなくなってしまいましたが、最近日程を変えて、火曜日の夜に始めました。暇な方一緒にもくもくしましょう。
- [強化学習の講義資料の翻訳](https://tadashik.github.io/rltheory-jp/)をしています。難しいですが、けっこう勉強になります。有限サンプルでのバウンドを、初めて勉強しました。興味がある方は、ぜひ一緒にやりましょう。
- 昨年の[ブログ](https://kngwyu.github.io/rlog/ja/2020/12/22/understanding-what-makes-rl-difficult.html)を書いてからはや一年、せっかく専用のブログを作ったので今年もいい感じに~~matplotlib芸~~強化学習の記事を書いていきたいと思っていましたが、結局何も書きませんでした。

最近は人工進化や人工生命の研究も始めたので、もはや「強化学習の人」と名乗っていいのかよくわかりませんが、今後も国内のコミュニティに何か貢献できればと思います。

今年は強化学習に対する楽観論も悲観論も多く目にした一年でした。
David SilverやSuttonは[Reward is Enough](https://www.sciencedirect.com/science/article/pii/S0004370221000862)という強気な論文を出し、知的なシステムはおよそ全て報酬最大化で作れると主張しました。
さすがに強気すぎると思いますが、その後[Reward is enough for convex MDPs](https://arxiv.org/abs/2106.00661)や[On the Expressivity of Markov Reward](https://openreview.net/forum?id=9DlCh34E1bN)といったテクニカルな論文が出てきたのは面白いです。
また、オフライン強化学習・教師なし強化学習の論文が増えてきたと思います。
ざっくり、**オフライン強化学習 = 強化学習 - 探索**、**教師なし強化学習 = 強化学習 - 報酬**と思ってもらって問題ないでしょう。
何を隠そう僕の修士論文も単なる「階層型強化学習」だったのですが、~~リジェクト~~諸般の事情により教師なしに魔改造して再投稿しました。
Sergey Levineにいたっては[Understanding the World Through Action](https://arxiv.org/abs/2110.12543)というタイトルが強い論文の中で、「大量にデータを集めてオフライン教師なし強化学習をすれば世界を理解できる（≒世界を理解しているのと同等のシステムが作れる？）」と言っています。面白い方向性だと思います。
一方で、みんな大好き[ルービックキューブ論文](https://openai.com/blog/solving-rubiks-cube/)を出したOpen AIのロボティクスチームは、「とりあえず今データがある領域に注力する」とのことで[解散](https://venturebeat.com/2021/07/16/openai-disbands-its-robotics-research-team/)してしまいました。
John Schulmanも[GPTを使った論文](https://arxiv.org/abs/2110.14168)を出していますし、今後は言語モデル＋強化学習で何か出てくるのでしょうか。品川さんが喜びそうですね。
Open AIはSim2Realにこだわりすぎたのでは？という意見を某所でお聞きしました。実際そうなのかは知りませんが、大規模にシミュレーションしてSim2Realを頑張るのか、実機のデータで頑張るのかというのは、面白い視点だと思います。

Open AIが今までほど強化学習に注力しなくなったことで、[Open AI gym](https://gym.openai.com/)をはじめ強化学習研究で使われてきたソフトウェア群にも、色々と情勢の変化がありそうです。
1. [OpenAI Gymのメンテナが変わりました](https://github.com/openai/gym/issues/2259)。これからはOpen AIではなくメリーランド大学の学生さんがメンテナになるようです。mujoco-pyなど関連するライブラリについては相変わらず放置されています。
2. [DeepmindがMuJoCoを買い取って無料にしました](https://deepmind.com/blog/announcements/mujoco)。今後ソースコードも公開されるようです。
3. Googleから新しく[brax](https://github.com/google/brax)というシミュレータが公開されました。

そんなわけで、僕はこれまでmujoco-py + gymで作成した[カスタム環境](https://github.com/kngwyu/mujoco-maze/)でたくさん実験をやってきましたが、MuJoCoを使うにしてもdm_controlを使うとか、はたまたbraxにしてしまうとか、別の選択肢を検討したくなってきました。
このブログでは、とりあえずbraxを試してみようと思います。

# 1. はじめに: シミュレーション・謎ロボット・GPU

本題に入りますが、ざっくり、強化学習とは、**報酬**から**行動**を学習する枠組みだと言うことができます。
では**何の**行動を学習させたいのでしょうか。
ゲームのAIだったり、チャットボットだったり、色々な選択肢があると思いますが、どういうわけか**シミュレータ上で動く謎ロボット**というのがポピュラーな選択肢です。

このブログをごらんの方の中には、こういったナナフシのような謎ロボットの画像を目にしたことがある方も多いのではないでしょうか。

<img src="https://gym.openai.com/videos/2019-10-21--mqt8Qj1mwo/HalfCheetah-v2/poster.jpg" alt="halfcheeta" width="300"/>

これはOpen AI gymのHalfCheetahというロボットです。足が2本なのでハーフなのだと思いますが、なんとも残酷なネーミングです。愛玩されるために病気のまま品種改良されてきた犬猫のような哀愁が漂います。

MuJoCoシミュレーターに「こことここがジョイントで、可動域はこうです。床は白黒でお願いします」みたいなXMLファイルを渡すと、こういうロボットを作ってくれます。
もしくは、[dm_control](https://github.com/deepmind/dm_control)などのPythonライブラリにXMLを作らせることもできます。
このような謎ロボットが実験で広く使われている要因として、
- みんなが使っているから
- Atariなどのゲームより高速
- ジョイントの速さ・位置などの完全な内部状態が手に入る
  - マルコフ性について心配しなくてもいい
- 色々カスタマイズできて便利だから
- 普通のロボットを訓練するためのテストにちょうどいいから
などの理由があると思いますが、なんだかんだみんなが使っているからというのが大きい気がします。

ところで、このMuJoCoシミュレータというのは非常に高速に動作するのですが、CPU上でしか動作しません。
今日使われている深層学習のコードは、その計算量のほとんどを占める行列演算がベクトル並列化ととても相性がいいため、ネットワークやバッチサイズが大きくなればなるほどGPU上で高速に動作します。
となると、GPUで学習を回している場合、どうしてもCPUからGPUにデータを転送するボトルネックが発生し、高速化の妨げになります。
そこで、GPU上でシミュレーションを行えるようにしたのが、今回紹介するbraxというシミュレータです。

# 2. Jaxでnumpy演算を高速化してみる

では、braxはCUDAか何かで書かれているのかな？と思ったかもしれませんが、なんと全てPythonで書かれているのです。
その鍵となるのが[jax](https://jax.readthedocs.io)というライブラリです。
おもむろに、インストールしてみましょう。